In [1]:
from dotenv import load_dotenv,find_dotenv
from langchain_core.prompts import PromptTemplate,ChatPromptTemplate
from langchain_core.messages import SystemMessage,HumanMessage,AIMessage
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI
from langchain.prompts import SystemMessagePromptTemplate,HumanMessagePromptTemplate

from langchain_core.example_selectors import MaxMarginalRelevanceExampleSelector,SemanticSimilarityExampleSelector
from langchain.prompts import PromptTemplate,FewShotPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.vectorstores import FAISS

In [2]:
load_dotenv(dotenv_path=find_dotenv(filename="../.env"))

True

In [3]:
llmGemini=ChatGoogleGenerativeAI(model="gemini-2.0-flash-001")
llmOpenAI=ChatOpenAI(model="gpt-3.5-turbo")

In [4]:
examples=[
    {"input":"happy","output":"sad"},
    {"input":"tall","output":"short"},
    {"input": "energetic", "output": "lethargic"},
    {"input": "sunny", "output": "gloomy"},
    {"input": "windy", "output": "calm"},
]

In [6]:
examplePrompt=PromptTemplate(
    input_variables=["input","output"],
    # The input and output are the input and output in the examples which gets mapped to the value of the input
    # So the format will be: "Input: sunny\n Output: gloomy
    template="Input: {input}\n Output:{output}",
    )

<h3>The MaxMarginalRelevanceExampleSelector selects examples based on a combination of which examples are most similar to the inputs, while also optimizing for Diversity. It does this by finding the examples with the embeddings that have the greatest cosine similarity with the inputs, and then iteratively adding them while penalizing them for closeness to already selected examples</h3>

In [9]:
exampleSelector=MaxMarginalRelevanceExampleSelector.from_examples(
    examples=examples,
    embeddings=SentenceTransformerEmbeddings(),
    vectorstore_cls=FAISS,
    k=2,  # Number of examples to select
)

C:\Users\MSI\AppData\Local\Temp\ipykernel_15092\248197981.py:3: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embeddings=SentenceTransformerEmbeddings(),


In [10]:
mmrPrompt=FewShotPromptTemplate(example_selector=exampleSelector,
                                prefix="Give the Antonym for every input",
                                example_prompt=examplePrompt,
                                suffix="Input Adjective: {adjective}\nAntonym: ",
                                input_variables=['adjective']
                               )

In [11]:
print(mmrPrompt.format(adjective="worried"))

Give the Antonym for every input

Input: happy
 Output:sad

Input: energetic
 Output:lethargic

Input Adjective: worried
Antonym: 


In [12]:
chain=RunnablePassthrough()|mmrPrompt|llmGemini|StrOutputParser()

In [13]:
print(chain.invoke(input={"adjective":"Untidy"}))

Antonym: Tidy


<h3>Semantic Similarity Example Selector</h3>

In [14]:
exampleSelector=SemanticSimilarityExampleSelector.from_examples(
    examples=examples,
    embeddings=SentenceTransformerEmbeddings(),
    vectorstore_cls=FAISS,
    k=2,  # Number of examples to select
)

C:\Users\MSI\AppData\Local\Temp\ipykernel_15092\319744806.py:3: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embeddings=SentenceTransformerEmbeddings(),


In [17]:
similarPrompt=FewShotPromptTemplate(example_selector=exampleSelector,
                                prefix="Give the Antonym for every input",
                                example_prompt=examplePrompt,
                                suffix="Input Adjective: {adjective}\nAntonym: ",
                                input_variables=['adjective']
                               )

In [18]:
print(similarPrompt.format(adjective="worried"))

Give the Antonym for every input

Input: happy
 Output:sad

Input: windy
 Output:calm

Input Adjective: worried
Antonym: 


In [19]:
chain=RunnablePassthrough()|similarPrompt|llmGemini|StrOutputParser()

In [20]:
print(chain.invoke(input={"adjective":"Untidy"}))

Antonym: Tidy
